In [171]:
# Code:
# https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/acgan/acgan.py

In [172]:
# Imports
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

# Models
from torchvision.models import vgg19
from sys import path
path.append("../utils")
from models import LeNet5

In [173]:
# Configs

cuda = False

n_epochs=200
batch_size=64
lr=0.0002
b1=0.5
b2=0.999
latent_dim=100
n_classes=10
img_size=28
channels=1
sample_interval=25

# target classifier conditional constants
threshold_valid = 0.65
threshold_classification = 0.6

lenet5_state_path = "../utils/models/trained_lenet5.pkl"


In [174]:
# Load functions for fixed target classifier

def load_pytorch_model():
    # Load model from torchvision.models (NOTE: VVG19 IS NOT FOR MNIST, DON'T USE)
    model = vgg19(pretrained=True)
    model.eval()
    if cuda:
        model.cuda()
    
    for p in model.features.parameters():
        p.requires_grad = False
    for p in model.classifier.parameters():
        p.requires_grad = False

    return model

def load_LeNet5():
    net = LeNet5()
    if cuda:
        net.cuda()
    
    # remove map location = cpu if using cuda
    net.load_state_dict(torch.load(lenet5_state_path, map_location=torch.device('cpu')))
    
    # set model to eval mode so nothing is changed
    net.eval()
    
    return net
    

In [175]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [176]:
# Models

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(n_classes, latent_dim)

        self.init_size = img_size // 4  # Initial size before upsampling
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise, labels):
        gen_input = torch.mul(self.label_emb(labels), noise)
        out = self.l1(gen_input)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            """Returns layers of each discriminator block"""
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.conv_blocks = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = 2#img_size // 2 ** 4

        # Output layers
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())
        self.aux_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, n_classes), nn.Softmax())

    def forward(self, img):
        out = self.conv_blocks(img)
        out = out.view(batch_size, -1)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)

        return validity, label


In [177]:
# Loss functions
adversarial_loss = nn.BCELoss()
auxiliary_loss = nn.CrossEntropyLoss()
target_classifier_loss = nn.CrossEntropyLoss() # negate target classifier output when passing to this loss function

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

# Load target classifier

target_classifier = load_LeNet5()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

In [178]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Configure data loader
os.makedirs("../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

In [179]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, "../images/%d.png" % batches_done, nrow=n_row, normalize=True)
    

In [180]:
# TEMPORARY SECTION: testing target classifier output

z = Variable(FloatTensor(np.random.normal(0, 1, (4 ** 2, latent_dim))))
labels = np.array([num for _ in range(4) for num in range(4)])
labels = Variable(LongTensor(labels))
imgs = generator(z, labels)
print(imgs[0].size())
output = target_classifier(imgs)
for o in output:
    print(o)


torch.Size([1, 28, 28])
tensor([-0.3920,  0.7273, -0.2902, -0.4444, -0.4154, -0.3107, -0.0947, -0.0616,
        -0.7260, -0.5456], grad_fn=<SelectBackward>)
tensor([-0.3823,  0.7064, -0.3269, -0.4879, -0.4277, -0.2601, -0.0635, -0.1159,
        -0.7006, -0.5161], grad_fn=<SelectBackward>)
tensor([-0.3834,  0.6984, -0.3338, -0.4996, -0.4231, -0.2465, -0.0513, -0.1272,
        -0.7072, -0.5075], grad_fn=<SelectBackward>)
tensor([-0.9914, -0.0766,  0.2700, -0.2436, -0.6665, -0.0363, -0.4535,  0.1195,
        -1.1724, -1.0688], grad_fn=<SelectBackward>)
tensor([-0.6083,  0.5284, -0.5882, -0.5825, -0.3515,  0.3027, -0.2871, -0.0588,
        -0.8580, -0.8378], grad_fn=<SelectBackward>)
tensor([-0.4431,  0.6382, -0.2500, -0.4825, -0.4389, -0.2284, -0.1140, -0.0621,
        -0.7684, -0.5679], grad_fn=<SelectBackward>)
tensor([-0.4444,  0.6348, -0.1535, -0.5722, -0.2626, -0.2298, -0.0665, -0.1607,
        -0.6166, -0.6565], grad_fn=<SelectBackward>)
tensor([-0.3920,  0.6880, -0.1478, -0.4001, -

In [181]:
# 
def shouldCalculateTargetLoss(validity, pred_label, label):
    # Only works if batch size is 1, so just uses the first value
    return (validity > threshold_valid and  # validity is above threshold
        pred_label[label] == np.amax(pred_label) and  # classification is correct
        pred_label[label] > threshold_classification)  # classification confidence is above threshold

def get_target_loss(validity, pred_labels, label, target_classification):
    # Apply conditional loss to each output individually
    loss = target_classifier_loss(target_classifier_pred_label * -1, gen_labels)
    loss_mult = torch.zeros(loss.size())
    for i in range(pred_labels.size(0)):
        if shouldCalculateTargetLoss(validity[i], pred_labels[i], label[i]):
            loss_mult[i] = 1
    
    print("Count of loss including target loss:", torch.sum(loss_mult))
    
    return loss * loss_mult

In [182]:
# Training
for epoch in range(n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):
        
        #batch_size = imgs.shape[0]
        
        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity, pred_label = discriminator(gen_imgs)
        target_classifier_pred_label = target_classifier(gen_imgs)
        g_loss = adversarial_loss(validity, valid) \
                    + auxiliary_loss(pred_label, gen_labels) \
                    + get_target_loss(validity, pred_label, gen_labels, target_classifier_pred_label)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        real_pred, real_aux = discriminator(real_imgs)
        d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

        # Loss for fake images
        fake_pred, fake_aux = discriminator(gen_imgs.detach())
        d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        # Calculate discriminator accuracy
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item())
        )
        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)


Count of loss including target loss: tensor(0.)
[Epoch 0/200] [Batch 0/938] [D loss: 1.497896, acc: 11%] [G loss: 2.987437]
Count of loss including target loss: tensor(0.)
[Epoch 0/200] [Batch 1/938] [D loss: 1.497943, acc: 13%] [G loss: 2.987810]
Count of loss including target loss: tensor(0.)
[Epoch 0/200] [Batch 2/938] [D loss: 1.497945, acc: 7%] [G loss: 2.988326]
Count of loss including target loss: tensor(0.)
[Epoch 0/200] [Batch 3/938] [D loss: 1.498060, acc: 7%] [G loss: 2.989039]
Count of loss including target loss: tensor(0.)
[Epoch 0/200] [Batch 4/938] [D loss: 1.497820, acc: 7%] [G loss: 2.988525]
Count of loss including target loss: tensor(0.)
[Epoch 0/200] [Batch 5/938] [D loss: 1.497743, acc: 14%] [G loss: 2.989434]
Count of loss including target loss: tensor(0.)
[Epoch 0/200] [Batch 6/938] [D loss: 1.497572, acc: 8%] [G loss: 2.989128]
Count of loss including target loss: tensor(0.)
[Epoch 0/200] [Batch 7/938] [D loss: 1.497858, acc: 9%] [G loss: 2.989874]
Count of loss

KeyboardInterrupt: 